In [1]:
import pandas as pd
from pathlib import Path

DATA_RAW_PATH = Path('..\\data\\raw')

#This dataset contains raw data from the original dataset where the rows containg missing values where dropped
data_raw_dropped = pd.read_csv(DATA_RAW_PATH/'version_dropped.csv', index_col='customerID')

# Models Based On Raw Data

## Version Missing Values Dropped

In [2]:
data_raw_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7010 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7010 non-null   object 
 1   SeniorCitizen     7010 non-null   int64  
 2   Partner           7010 non-null   object 
 3   Dependents        7010 non-null   object 
 4   tenure            7010 non-null   int64  
 5   PhoneService      7010 non-null   object 
 6   MultipleLines     7010 non-null   object 
 7   InternetService   7010 non-null   object 
 8   OnlineSecurity    7010 non-null   object 
 9   OnlineBackup      7010 non-null   object 
 10  DeviceProtection  7010 non-null   object 
 11  TechSupport       7010 non-null   object 
 12  StreamingTV       7010 non-null   object 
 13  StreamingMovies   7010 non-null   object 
 14  Contract          7010 non-null   object 
 15  PaperlessBilling  7010 non-null   object 
 16  PaymentMethod     7010 non-null 

In [3]:
X = data_raw_dropped.drop('Churn', axis=1)
y = data_raw_dropped['Churn']

In [27]:
numerical_col = X.select_dtypes('number').columns
categorical_col = X.select_dtypes(exclude='number').columns
target_col = y.name

In [5]:
numerical_col[1: ]

Index(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [6]:
categorical_col

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [28]:
target_col

'Churn'

In [33]:
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler

inputs_processor = make_column_transformer(
    (OrdinalEncoder(), categorical_col),
    (StandardScaler(), numerical_col)
)

target_enc = LabelEncoder()

raw_data_pipeline = Pipeline([('preprocessing raw data', inputs_processor)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

X_train = raw_data_pipeline.fit_transform(X_train)
y_train = target_enc.fit_transform(y_train)

array(['No', 'Yes'], dtype=object)